# インプライド・ボラティリティ（IV）の計算方法

インプライド・ボラティリティ（IV）とは、市場価格を与えたときに、その価格をブラック・ショールズ式で再現するボラティリティ のことです。

つまり、市場が「このオプションはこのボラティリティで取引されている」と示す値です。

IV は、ブラック・ショールズモデルを**逆算**して求めます。
ブラック・ショールズ式は、オプション価格をボラティリティ sigma の関数として表します：

C=BS(S0,K,T,r,σ)

ここで、

- C ：市場で観測されたオプションの価格（実際の取引価格）
- BS(S0,K,T,r,σ)：ブラック・ショールズ方程式
- σ：ボラティリティ（これを逆算したい）
- 
IV を求めるには、ブラック・ショールズ式を満たす sigma を探す 必要があります。

これは**数式的に解けないため、数値解法（ニュートン法など） で解きます。**

## ブラック・ショールズ方程式

ヨーロピアン・コールオプションの価格 $C$ は、以下の式で与えられる：

$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$

ヨーロピアン・プットオプションの価格 \(P\) は、以下の式で与えられる：

$
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
$

ここで、

$
d_1 = \frac{\ln(S_0 / K) + (r + \frac{1}{2} \sigma^2) T}{\sigma \sqrt{T}}
$

$
d_2 = d_1 - \sigma \sqrt{T}
$

### 変数の説明
- $S_0$ ：現在の株価（原資産価格）
- $K$ ：権利行使価格（行使価格）
- $T$ ：満期までの時間（年）
- $r$ ：無リスク金利（リスクフリーレート）
- $\sigma$ ：ボラティリティ（原資産の標準偏差）
- $N(x)$ ：標準正規分布の累積分布関数（CDF）



In [16]:
from scipy.stats import norm
import numpy as np
from scipy.optimize import bisect

def black_scholes(S0, K, T, r, sigma, option_type="call"):
    """ブラック・ショールズモデルでオプション価格を計算"""
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
    return price

def implied_volatility(market_price, S0, K, T, r, option_type="call"):
    """市場価格からインプライドボラティリティを計算"""
    def objective(sigma):
        return black_scholes(S0, K, T, r, sigma, option_type) - market_price
    
    # 最小・最大のボラティリティでの価格
    min_price = black_scholes(S0, K, T, r, 0.001, option_type)
    max_price = black_scholes(S0, K, T, r, 5.0, option_type)
    
    # 市場価格が範囲外の場合は NaN を返す
    if market_price < min_price or market_price > max_price:
        print(f"市場価格 {market_price} は {min_price:.2f} 〜 {max_price:.2f} の範囲外です。")
        return np.nan
    
    try:
        iv = bisect(objective, 0.001, 5.0, xtol=1e-6)
        return iv
    except ValueError:
        print("ボラティリティの求解に失敗しました。")
        return np.nan  # 解が見つからない場合

# パラメータ設定
S0 = 100    # 現在の株価
K = 150     # 権利行使価格
T = 1       # 満期 (年)
r = 0.05    # リスクフリーレート
market_price = 10.0  # 市場価格

# インプライドボラティリティ計算
iv = implied_volatility(market_price, S0, K, T, r, option_type="call")
if not np.isnan(iv):
    print(f"インプライドボラティリティ: {iv * 100:.2f}%")


インプライドボラティリティ: 54.84%


In [17]:

# 理論価格をインプライドボラティリティを使って計算
theoretical_price = black_scholes(S0, K, T, r, iv, option_type='call')

# 市場価格と理論価格を比較
print(f"理論価格: {theoretical_price:.2f}")
print(f"市場価格: {market_price:.2f}")

理論価格: 10.00
市場価格: 10.00


✅ まとめ
- IV はブラック・ショールズ式を逆算して求める
- 数値解法（scipy.optimize.bisect）で解く
- IV を使えば、市場のボラティリティ予測やオプションの割高・割安を判断できる

# 数値解法（Numerical Methods）とは

解析的な方法で解けないまたは解きにくい問題を近似的に解くための数学的技法です。これには、方程式や微分方程式の数値的解法、最適化問題、積分の近似などが含まれます。特に金融工学や経済学において、数値解法は非常に重要であり、ブラック-ショールズ方程式やオプション価格の計算でよく使用されます。

インプライドボラティリティ（Implied Volatility）は、オプション価格に基づいて計算されるボラティリティで、実際の市場で取引されているオプションの価格から逆算されます。具体的には、ブラック-ショールズモデルに基づいて、オプションの市場価格と理論価格（ブラック-ショールズモデルを使って計算された価格）の差を最小にするボラティリティを求めます。

ブラック-ショールズ方程式を使ってオプションの理論価格を計算する場合、理論価格はインプライドボラティリティに依存しますが、このボラティリティは解析的に求めることができません。したがって、インプライドボラティリティを求めるために数値解法が用いられます。

たとえば、ブラック-ショールズ方程式において、オプション価格とボラティリティの関係は非線形です。これを解くためには、数値的な方法（ニュートン法や二分法など）を用いて、特定のオプション価格に対するボラティリティを反復的に求めることになります。インプライドボラティリティは、次のようにして求めます：

オプション市場での実際の取引価格を取得。
ブラック-ショールズ方程式を用いて理論価格を計算。
理論価格と市場価格の差を最小化するボラティリティを反復的に求める（数値解法を使用）。
このプロセスは、解析的な解法が存在しないため、数値解法が非常に重要になります。

このコードは、ブラック-ショールズ方程式を用いてオプションの理論価格を計算し、実際の市場価格と比較することで、インプライドボラティリティ（IV）を求めるためのものです。その意義は以下の通りです。

1. オプション価格の理論的な根拠を提供

ブラック-ショールズモデルは、オプション価格を決定するための理論的枠組みを提供します。市場で取引されるオプション価格は、実際には供給と需要によって決まりますが、このコードでは、理論的にどのような価格になるべきかを計算しています。これにより、オプションが市場でどれほど「適正」に取引されているかを分析することができます。

2. インプライドボラティリティ（IV）の求解

インプライドボラティリティ（IV）は、オプション市場の最も重要なパラメータの1つで、オプションの理論価格が市場価格と一致するようなボラティリティです。通常、IVは市場価格から逆算する必要があり、解析的な解法は存在しません。そのため、数値解法（この場合、fsolve）を用いてIVを求めることになります。このプロセスは、ブラック-ショールズ方程式が提供する理論価格と市場価格のギャップを最小化するボラティリティを見つけることを意味します。

3. 市場の過小評価・過大評価を分析
   
インプライドボラティリティを求めることで、市場がオプションのリスクをどれほど評価しているかが明らかになります。具体的には：

- 市場価格が理論価格より高い場合：オプションが過大評価されている可能性があり、投資家は将来の価格変動（ボラティリティ）が非常に大きいと予測しているか、需要が過剰であることを示唆します。
  
- 市場価格が理論価格より低い場合：オプションが過小評価されている可能性があり、投資家は将来の価格変動を過小評価しているか、市場で取引されている価格が低すぎることを示唆します。
このコードを使って、実際に取引されているオプションの価格と理論的に計算された価格を比較し、その差が意味するところを分析できます。

4. オプション市場の効率性の評価

インプライドボラティリティの計算を通じて、オプション市場がどれほど効率的かを評価することができます。理論価格と市場価格に大きな差がある場合、市場は効率的でないと考えられ、そのギャップを取引機会として利用できる場合があります。特に、オプション取引におけるアービトラージ（価格差を利用した無リスク取引）の可能性を分析する際に有用です。

5. リスク管理とポートフォリオ最適化

インプライドボラティリティはオプション取引におけるリスク管理やポートフォリオ最適化に役立ちます。ボラティリティが高ければ、オプションのリスクも大きくなるため、投資家はそのリスクに応じたポートフォリオの調整を行います。IVを正確に求めることで、オプションの価格とリスクの関係をより深く理解でき、効果的なヘッジ戦略を立てることが可能になります。

6. オプション取引における意思決定をサポート

実際のオプション取引において、このコードを使用することで、投資家やトレーダーは市場での取引判断をより明確に行うことができます。インプライドボラティリティを求めることで、市場がどれほどオプションに対するボラティリティを評価しているかを理解し、それに基づいた戦略（例えば、売買タイミングの判断）を構築する手助けとなります。

# 理論価格（Black-Scholesモデルで算出した価格）と市場価格が完全に一致する場合

それは 市場がBlack-Scholesモデルの仮定を完全に反映している ことを意味します。具体的には、以下のような状況が考えられます。

1. ボラティリティが市場の期待と一致している
インプライドボラティリティ（IV） と ヒストリカルボラティリティ（HV, 過去の実績ボラティリティ） がほぼ同じ
つまり、市場参加者がオプション価格を決める際に想定しているボラティリティと、Black-Scholesモデルが計算するボラティリティが一致している
2. アービトラージの余地がない（裁定機会がない）
市場の価格形成が効率的であり、過大評価・過小評価されているオプションがない
アービトラージャーが介入する必要がなく、市場が均衡状態にある
3. Black-Scholesモデルの前提が市場で成り立っている
Black-Scholesモデルは以下のような仮定に基づいている：

ボラティリティが一定（実際の市場ではボラティリティは変動するが、それが一定だと仮定できる）
市場に摩擦（取引コスト、スプレッド、税金など）がない
金利が一定で借入・貸出が自由にできる
株価は幾何ブラウン運動に従う（ジャンプやボラティリティ・スマイルなし）
これらの仮定が市場の実態に近ければ、理論価格と市場価格は一致しやすくなる。

# 逆に、一致しない場合はどうなるか？
市場価格が理論価格と異なる場合、通常は市場参加者がモデルの前提とは異なる何かを考慮していることを意味します。例えば：

市場のボラティリティが変動すると予想されている
ボラティリティ・スマイルやスキューが存在する
流動性の問題がある（取引量が少なく価格が歪んでいる）
市場の需給によるプレミアム（買いが多く価格が上がるなど）
大口投資家の影響（ヘッジファンドや機関投資家のオプション戦略）